# Usage of the Mitigator object (PyQuil)

This notebook shows the usage of the `run_mitigation` function and the associated `fun_mitigator` decorator.
 
**Important: We assume that the user is able to evaluate the expectation value of an input circuit and also to increase the noise level of the backend.**

In [1]:
import numpy as np

# Error mitigation
from mitiq import run_mitigation, fun_mitigator

In [2]:
# PyQuil
from pyquil import get_qc, Program
from pyquil.gates import X,Y,Z, MEASURE

# Noise simulation
from pyquil.noise import append_kraus_to_gate

The following function generates a random sequence of Pauli gates which is equivalent to the identity.

In [3]:
def random_identity_circuit(depth=None):
    """Returns a single-qubit identity circuit based on Pauli gates."""
    
    # initialize a quantum circuit 
    prog = Program()  
    
    # index of the (inverting) final gate: 0=I, 1=X, 2=Y, 3=Z
    k_inv = 0

    # apply a random sequence of Pauli gates
    for _ in range(depth):
        # random index for the next gate: 1=X, 2=Y, 3=Z
        k = np.random.choice([1, 2, 3])
        # apply the Pauli gate "k"
        if k == 1:
            prog += X(0)
        elif k == 2:
            prog += Y(0)
        elif k == 3:
            prog += Z(0)
        
        # update the inverse index according to 
        # the product rules of Pauli matrices k and k_inv
        if k_inv == 0:
            k_inv = k
        elif k_inv == k:
            k_inv = 0
        else:
            _ = [1, 2, 3]
            _.remove(k_inv)
            _.remove(k)
            k_inv = _[0]
    
    # apply the final inverse gate
    if k_inv == 1:
        prog += X(0)
    elif k_inv == 2:
        prog += Y(0)
    elif k_inv == 3:
        prog += Z(0)
        
    return prog

We assume that the user is able to define a function which:
* takes as arguments a circuit and a noise stretch factor,
* executes the circuit in an arbitrary backend,
* returns the final expectation value.

In [4]:
def noisy_run(circuit=None, stretch=1):
    """Execute a circuit on a noisy device and returns the expectation value of 
    the final measurement."""
    
    _circuit = circuit.copy()
    
    shots = 10 ** 3      # measurement shots
    true_noise = 0.007    # real value of the noise
    
    # noise stretching
    noise = true_noise * stretch

    # apply depolarizing noise to all gates
    npI = np.array([[1, 0], [0, 1]])
    npX = np.array([[0, 1], [1, 0]])
    npY = np.array([[0, -1j], [1j, 0]])
    npZ = np.array([[1, 0], [0, -1]])
    kraus_ops = [np.sqrt(1 - noise) * npI, 
                     np.sqrt(noise / 3) * npX,
                     np.sqrt(noise / 3) * npY,
                     np.sqrt(noise / 3) * npZ]
    _circuit.define_noisy_gate("X", [0],  append_kraus_to_gate(kraus_ops, npX))
    _circuit.define_noisy_gate("Y", [0],  append_kraus_to_gate(kraus_ops, npY))
    _circuit.define_noisy_gate("Z", [0],  append_kraus_to_gate(kraus_ops, npZ))


    # set number of shots
    _circuit.wrap_in_numshots_loop(shots)
    
    # we want to simulate noise, so we run without compiling
    
    results = qvm.run(_circuit)
    expval = (results == [0]).sum() / shots
    return expval

Let us use the function without error mitigation:

In [5]:
qvm = get_qc('1q-qvm')

# add measurement
rand_circ = random_identity_circuit(depth=30)
ro = rand_circ.declare('ro', 'BIT', 1)
rand_circ += MEASURE(0, ro[0])

# execution
noisy_run(rand_circ)

0.74593

Now we use the error the mitigation toolkit (without using the decorator):

In [6]:
qvm = get_qc('1q-qvm')

# add measurement
rand_circ = random_identity_circuit(depth=30)
ro = rand_circ.declare('ro', 'BIT', 1)
rand_circ += MEASURE(0, ro[0])

# execution with automatic error mitigation
run_mitigation(noisy_run, rand_circ, order=2)

0.9351400000000001

The user can also directly decorate its own function with a single line of code `@fun_mitigator()`, placed right above the function definition. For example:

In [7]:
@fun_mitigator(order=2)
def magic_run(circuit=None, stretch=1):
     """Execute a circuit on a noisy device and returns the expectation value of 
    the final measurement."""
    
    _circuit = circuit.copy()
    
    shots = 10 ** 3      # measurement shots
    true_noise = 0.007    # real value of the noise
    
    # noise stretching
    noise = true_noise * stretch

    # apply depolarizing noise to all gates
    npI = np.array([[1, 0], [0, 1]])
    npX = np.array([[0, 1], [1, 0]])
    npY = np.array([[0, -1j], [1j, 0]])
    npZ = np.array([[1, 0], [0, -1]])
    kraus_ops = [np.sqrt(1 - noise) * npI, 
                     np.sqrt(noise / 3) * npX,
                     np.sqrt(noise / 3) * npY,
                     np.sqrt(noise / 3) * npZ]
    _circuit.define_noisy_gate("X", [0],  append_kraus_to_gate(kraus_ops, npX))
    _circuit.define_noisy_gate("Y", [0],  append_kraus_to_gate(kraus_ops, npY))
    _circuit.define_noisy_gate("Z", [0],  append_kraus_to_gate(kraus_ops, npZ))


    # set number of shots
    _circuit.wrap_in_numshots_loop(shots)
    
    # we want to simulate noise, so we run without compiling
    
    results = qvm.run(_circuit)
    expval = (results == [0]).sum() / shots
    return expval

Now the user can directly apply the function `magic_run` to evaluate circuits. Error mitigation is automatically applied behind the scenes!

In [8]:
qvm = get_qc('1q-qvm')

# add measurement
rand_circ = random_identity_circuit(depth=30)
ro = rand_circ.declare('ro', 'BIT', 1)
rand_circ += MEASURE(0, ro[0])

# execution with automatic error mitigation
noisy_run(rand_circ)

0.9362800000000002